<a href="https://colab.research.google.com/github/Gurjot-Singh-2002/UCS645-Lab-Assignments/blob/main/Assignment%204/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gurjot Singh 102203582 3CO14

#Program 1

In [1]:
%%writefile sumofn.cu
#include <stdio.h>
#include <cuda_runtime.h>

#define N 1024  // Number of integers to sum

// Kernel function where each thread performs a different task
__global__ void sumKernel(int *input, int *output) {
    int tid = threadIdx.x;  // Thread ID within the block

    if (tid == 0) {
        // Task A: Iterative Sum
        int sum = 0;
        for (int i = 0; i < N; i++) {
            sum += input[i];  // Sum elements from input array
        }
        output[0] = sum;  // Store result in output[0]
    }
    else if (tid == 1) {
        // Task B: Direct Formula Sum
        output[1] = (N * (N - 1)) / 2;  // Using formula: n(n-1)/2
    }
}

int main() {
    int h_input[N], h_output[2] = {0};  // Host arrays

    // Fill input array with first N integers: 0 to N-1
    for (int i = 0; i < N; i++) {
        h_input[i] = i;
    }

    int *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(int));        // Allocate memory on device
    cudaMalloc(&d_output, 2 * sizeof(int));       // Output for both methods

    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);  // Copy input to device

    // Timing using CUDA events
    cudaEvent_t start, stop;
    float milliseconds = 0;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);  // Start recording time

    // Launch kernel with 1 block and 2 threads (each does a different task)
    sumKernel<<<1, 2>>>(d_input, d_output);

    cudaEventRecord(stop);   // Stop recording time
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);  // Calculate elapsed time

    // Copy results back from device to host
    cudaMemcpy(h_output, d_output, 2 * sizeof(int), cudaMemcpyDeviceToHost);

    // Print both results
    printf("Sum using Iteration (Thread 0): %d\n", h_output[0]);
    printf("Sum using Formula   (Thread 1): %d\n", h_output[1]);
    printf("GPU Kernel Execution Time: %.6f ms\n", milliseconds);

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Writing sumofn.cu


Program Compilation---

In [2]:
!nvcc --gpu-architecture=sm_70 sumofn.cu -o sumofn

Program Execution---

In [3]:
!./sumofn

Sum using Iteration (Thread 0): 523776
Sum using Formula   (Thread 1): 523776
GPU Kernel Execution Time: 0.147424 ms


#Program 2(a)

In [4]:
%%writefile pipelined_merge_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define N 1000  // Size of the array to be sorted

// Merge two sorted subarrays into one
void merge(int arr[], int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;  // Length of left subarray
    int n2 = r - m;      // Length of right subarray

    // Temporary arrays
    int *L = (int *)malloc(n1 * sizeof(int));
    int *R = (int *)malloc(n2 * sizeof(int));

    // Copy data to temp arrays
    for (i = 0; i < n1; i++) L[i] = arr[l + i];
    for (j = 0; j < n2; j++) R[j] = arr[m + 1 + j];

    i = j = 0;
    k = l;

    // Merge L and R back into arr
    while (i < n1 && j < n2)
        arr[k++] = (L[i] <= R[j]) ? L[i++] : R[j++];

    // Copy remaining elements
    while (i < n1) arr[k++] = L[i++];
    while (j < n2) arr[k++] = R[j++];

    // Free temporary arrays
    free(L);
    free(R);
}

// Recursive merge sort
void mergeSort(int arr[], int l, int r) {
    if (l < r) {
        int m = l + (r - l) / 2;  // Midpoint
        mergeSort(arr, l, m);     // Sort left half
        mergeSort(arr, m + 1, r); // Sort right half
        merge(arr, l, m, r);      // Merge halves
    }
}

int main() {
    int arr[N];

    // Fill array with random numbers between 0 and 999
    for (int i = 0; i < N; i++)
        arr[i] = rand() % 1000;

    // Record start time
    clock_t start = clock();

    // Perform merge sort
    mergeSort(arr, 0, N - 1);

    // Record end time
    clock_t end = clock();

    // Calculate elapsed time in milliseconds
    double time_taken = ((double)(end - start)) / CLOCKS_PER_SEC * 1000.0;

    // Print first 10 elements of the sorted array
    printf("Sorted array (first 10):\n");
    for (int i = 0; i < 10; i++)
        printf("%d ", arr[i]);
    printf("\nCPU Execution Time: %.4f ms\n", time_taken);

    return 0;
}


Writing pipelined_merge_sort.c


Program Compilation---

In [5]:
!nvcc --gpu-architecture=sm_70 pipelined_merge_sort.c -o pipelined_merge_sort

Porgram Execution---

In [6]:
!./pipelined_merge_sort

Sorted array (first 10):
0 0 2 2 4 6 8 9 10 11 
CPU Execution Time: 0.2030 ms


#Program 2(b)

In [7]:
%%writefile cuda_merge_sort.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <time.h>

#define N 1000  // Number of elements to sort

// Merge function for device - merges two sorted halves
__device__ void merge(int *arr, int l, int m, int r, int *temp) {
    int i = l, j = m + 1, k = l;

    // Merge elements into temp array
    while (i <= m && j <= r)
        temp[k++] = (arr[i] <= arr[j]) ? arr[i++] : arr[j++];

    // Copy remaining elements (if any)
    while (i <= m) temp[k++] = arr[i++];
    while (j <= r) temp[k++] = arr[j++];

    // Copy back from temp to original array
    for (i = l; i <= r; i++) arr[i] = temp[i];
}

// CUDA kernel to merge pairs of subarrays of given width
__global__ void mergeSortKernel(int *d_arr, int *d_temp, int width) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int left = tid * (2 * width);           // Start index of subarray
    int mid = left + width - 1;             // Middle index
    int right = left + 2 * width - 1;       // End index

    if (right < N)
        merge(d_arr, left, mid, right, d_temp);
    else if (mid < N)
        merge(d_arr, left, mid, N - 1, d_temp);
}

// Utility to check CUDA errors
void checkCuda(cudaError_t err, const char *msg) {
    if (err != cudaSuccess) {
        printf("CUDA error at %s: %s\n", msg, cudaGetErrorString(err));
        exit(-1);
    }
}

int main() {
    int *h_arr = (int*)malloc(N * sizeof(int));  // Host array
    int *d_arr, *d_temp;                         // Device arrays

    // Fill host array with random integers
    for (int i = 0; i < N; i++) h_arr[i] = rand() % 1000;

    // Allocate memory on device
    checkCuda(cudaMalloc(&d_arr, N * sizeof(int)), "cudaMalloc d_arr");
    checkCuda(cudaMalloc(&d_temp, N * sizeof(int)), "cudaMalloc d_temp");

    // Copy host array to device
    checkCuda(cudaMemcpy(d_arr, h_arr, N * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy H2D");

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);  // Start timing

    // Launch merge sort kernel iteratively increasing width
    for (int width = 1; width < N; width *= 2) {
        int numThreads = (N + (2 * width - 1)) / (2 * width);
        mergeSortKernel<<<(numThreads + 255)/256, 256>>>(d_arr, d_temp, width);
        checkCuda(cudaGetLastError(), "Kernel");
        checkCuda(cudaDeviceSynchronize(), "Sync");
    }

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);  // Stop timing

    // Get elapsed time in ms
    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    // Copy sorted array back to host
    checkCuda(cudaMemcpy(h_arr, d_arr, N * sizeof(int), cudaMemcpyDeviceToHost), "Memcpy D2H");

    // Print first 10 sorted values
    printf("Sorted array (first 10):\n");
    for (int i = 0; i < 10; i++) printf("%d ", h_arr[i]);
    printf("\nExecution time (GPU): %.4f ms\n", ms);

    // Free memory
    cudaFree(d_arr);
    cudaFree(d_temp);
    free(h_arr);

    return 0;
}


Writing cuda_merge_sort.cu


Program Compilation---

In [8]:
!nvcc --gpu-architecture=sm_70 cuda_merge_sort.cu -o cuda_merge_sort

Program Execution---

In [9]:
!./cuda_merge_sort

Sorted array (first 10):
0 0 2 2 4 6 8 9 10 11 
Execution time (GPU): 1.2908 ms
